In [2]:
import os
import smtplib
import logging
import psycopg2
from email.message import EmailMessage
from dotenv import load_dotenv
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv()

def validate_config():
    """Validate required environment variables"""
    required_vars = [
        "EMAIL_ADDRESS", "EMAIL_PASSWORD", "RECIPIENT_EMAIL",
        "DB_HOST", "DB_NAME", "DB_USER", "DB_PASSWORD"
    ]
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    if missing_vars:
        raise ValueError(f"Missing environment variables: {', '.join(missing_vars)}")

def get_weather_data(city="Nairobi", days=7):
    """Fetch weather data from database"""
    DB_CONFIG = {
        "host": os.getenv("DB_HOST"),
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": os.getenv("DB_PORT", 5432)
    }
    
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                # Get detailed records
                cur.execute('''
                    SELECT timestamp, description, temperature, humidity, pressure, wind_speed
                    FROM weather_data
                    WHERE city = %s AND timestamp > NOW() - INTERVAL %s
                    ORDER BY timestamp
                ''', (city, f"{days} days"))
                records = cur.fetchall()

                # Get summary statistics
                cur.execute('''
                    SELECT 
                        AVG(temperature) as avg_temp,
                        MIN(temperature) as min_temp,
                        MAX(temperature) as max_temp,
                        AVG(humidity) as avg_humidity
                    FROM weather_data
                    WHERE city = %s AND timestamp > NOW() - INTERVAL %s
                ''', (city, f"{days} days"))
                stats = cur.fetchone()
                
        return records, stats
        
    except psycopg2.Error as e:
        logger.error(f"Database error: {e}")
        raise

def create_email_content(records, stats, city="Nairobi"):
    """Generate email content with text and HTML versions"""
    if not records:
        return {
            "text": f"No weather data available for {city} in the last 7 days.",
            "html": f"<p>No weather data available for {city} in the last 7 days.</p>"
        }
    
    # Text version
    text_content = f"📊 Weekly Weather Summary for {city}:\n\n"
    text_content += f"🌡️ Temperature: Avg {stats[0]:.1f}°C (Min: {stats[1]}°C, Max: {stats[2]}°C)\n"
    text_content += f"💧 Humidity: Avg {stats[3]:.1f}%\n\n"
    text_content += "Detailed Records:\n"
    for ts, desc, temp, hum, pres, wind in records:
        text_content += f"{ts.strftime('%Y-%m-%d %H:%M')} - {desc.title()}, Temp: {temp}°C, Humidity: {hum}%\n"

    # HTML version
    html_content = f"""
    <h1>📊 Weekly Weather Summary for {city}</h1>
    <p><strong>🌡️ Temperature:</strong> Avg {stats[0]:.1f}°C (Min: {stats[1]}°C, Max: {stats[2]}°C)</p>
    <p><strong>💧 Humidity:</strong> Avg {stats[3]:.1f}%</p>
    <h2>Detailed Records:</h2>
    <ul>
    """
    for ts, desc, temp, hum, pres, wind in records:
        html_content += f"""
        <li>
            {ts.strftime('%Y-%m-%d %H:%M')} - {desc.title()}, 
            Temp: {temp}°C, Humidity: {hum}%, 
            Pressure: {pres} hPa, Wind: {wind} m/s
        </li>
        """
    html_content += "</ul>"
    
    return {"text": text_content, "html": html_content}

def send_email(content, subject):
    """Send email with both text and HTML content"""
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = os.getenv("EMAIL_ADDRESS")
    msg["To"] = os.getenv("RECIPIENT_EMAIL")
    
    msg.set_content(content["text"])
    msg.add_alternative(content["html"], subtype="html")
    
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(os.getenv("EMAIL_ADDRESS"), os.getenv("EMAIL_PASSWORD"))
            smtp.send_message(msg)
        logger.info("Email sent successfully")
    except smtplib.SMTPException as e:
        logger.error(f"Failed to send email: {e}")
        raise

def send_weekly_summary_email():
    """Main function to send weekly summary"""
    try:
        validate_config()
        records, stats = get_weather_data()
        content = create_email_content(records, stats)
        send_email(content, "Weekly Weather Summary - Nairobi")
    except Exception as e:
        logger.error(f"Failed to send weekly summary: {e}")

if __name__ == "__main__":
    send_weekly_summary_email()

INFO:__main__:Email sent successfully
